## Preprocess of the WISDM dataset

- Loading the initial dataset
- Changing the order of the columns
- Renaming the columns
- Making the labels from strings to integers
- Divide the dataset into datasets for each user
- Cleaning the data
- Lowpass filter
- Calculating the acceleration magnitude as feature
- Splitting the data into windows and features
- Saving the datasets per users and concatenated


In [17]:
# Importing libraries needed
import os
import sys

import pandas as pd

sys.path.append(os.path.abspath(os.path.join("../src/utils/")))
import feature_extraction
import preprocessing

In [18]:
# Read in the raw data
df = pd.read_csv(
    r"../data/activity_recognition_wisdm/time_series_data_human_activities.csv"
)
df.head()

,user,activity,timestamp,x-axis,y-axis,z-axis
0,1,Walking,4991922345000,0.69,10.80,-2.03
1,1,Walking,4991972333000,6.85,7.44,-0.50
2,1,Walking,4992022351000,0.93,5.63,-0.50
3,1,Walking,4992072339000,-2.11,5.01,-0.69
4,1,Walking,4992122358000,-4.59,4.29,-1.95


In [19]:
# Checking the basic info of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1073623 entries, 0 to 1073622
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   user       1073623 non-null  int64  
 1   activity   1073623 non-null  object 
 2   timestamp  1073623 non-null  int64  
 3   x-axis     1073623 non-null  float64
 4   y-axis     1073623 non-null  float64
 5   z-axis     1073623 non-null  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 49.1+ MB


In [20]:
# Change the column order
df = df[["user", "timestamp", "x-axis", "y-axis", "z-axis", "activity"]]
df.head()

,user,timestamp,x-axis,y-axis,z-axis,activity
0,1,4991922345000,0.69,10.80,-2.03,Walking
1,1,4991972333000,6.85,7.44,-0.50,Walking
2,1,4992022351000,0.93,5.63,-0.50,Walking
3,1,4992072339000,-2.11,5.01,-0.69,Walking
4,1,4992122358000,-4.59,4.29,-1.95,Walking


In [21]:
# Rename the columns to be universal
df.columns = [
    "User_ID",
    "Timestamp",
    "Acc.X.Phone",
    "Acc.Y.Phone",
    "Acc.Z.Phone",
    "Label",
]

In [22]:
# Check the unique values of the label
df.Label.unique()

array(['Walking', 'Jogging', 'Upstairs', 'Downstairs', 'Sitting',
       'Standing'], dtype=object)

In [23]:
# Map the labels to numbers
map_dict = {
    "Walking": 1,
    "Jogging": 2,
    "Upstairs": 3,
    "Downstairs": 4,
    "Sitting": 5,
    "Standing": 6,
}
df["Label"] = df["Label"].map(map_dict)

In [24]:
# Checking the number of unique users
df.User_ID.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36], dtype=int64)

In [25]:
# Dividing the data by user_id
user_id_dataframes = []
for user_id in df.User_ID.unique():
    user_id_dataframes.append(df[df.User_ID == user_id])

In [26]:
# Checking if everything is correct
user_id_dataframes[0].head()

,User_ID,Timestamp,Acc.X.Phone,Acc.Y.Phone,Acc.Z.Phone,Label
0,1,4991922345000,0.69,10.80,-2.03,1
1,1,4991972333000,6.85,7.44,-0.50,1
2,1,4992022351000,0.93,5.63,-0.50,1
3,1,4992072339000,-2.11,5.01,-0.69,1
4,1,4992122358000,-4.59,4.29,-1.95,1


In [27]:
# Setting the parameters for the preprocessing
sampling_frequency = 20
cutoff_frequency = 5
order = 3

In [28]:
# Lowpass filter the sensor data and calculate the magnitude
col_signals = ["Acc.X.Phone", "Acc.Y.Phone", "Acc.Z.Phone"]
for user_id_dataframe in user_id_dataframes:
    for col_signal in col_signals:
        user_id_dataframe[col_signal] = preprocessing.lowpass_filter(
            user_id_dataframe[col_signal], sampling_frequency, cutoff_frequency, order
        )
    user_id_dataframe = preprocessing.calculate_mag(user_id_dataframe, col_signals)
    labels = user_id_dataframe["Label"]
    user_id_dataframe.drop("Label", axis=1, inplace=True)
    user_id_dataframe["Label"] = labels

C:\Users\jakim\AppData\Local\Temp\ipykernel_10864\3869356920.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_id_dataframe[col_signal] = preprocessing.lowpass_filter(user_id_dataframe[col_signal], sampling_frequency, cutoff_frequency, order)
d:\Work\HAR-Comparison-Privacy\src\utils\preprocessing.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[mag_col_name] = mag
C:\Users\jakim\AppData\Local\Temp\ipykernel_10864\3869356920.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [29]:
# Checking if everything is correct
user_id_dataframes[0].head()

,User_ID,Timestamp,Acc.X.Phone,Acc.Y.Phone,Acc.Z.Phone,Acc.Magnitude.Phone,Label
0,1,4991922345000,0.690247,10.799816,-2.029869,11.010578,1
1,1,4991972333000,4.964224,7.771950,-0.443757,9.232749,1
2,1,4992022351000,1.626367,5.242737,-1.027593,5.584560,1
3,1,4992072339000,-3.656171,4.476615,-1.484754,5.967593,1
4,1,4992122358000,-0.008137,6.326895,1.499345,6.502131,1


In [30]:
# Setting the parameters for the feature extraction
window_duration = 0.3
overlap = 0.3
win_length = int(window_duration * sampling_frequency)
overlap = int(overlap * sampling_frequency)
col_extract = ["Acc.X.Phone", "Acc.Y.Phone", "Acc.Z.Phone", "Acc.Magnitude.Phone"]

In [31]:
# Extracting the features and generating the labels for each user
new_user_id_dataframes = []
for user_id_dataframe in user_id_dataframes:
    features = feature_extraction.calculate_features(
        user_id_dataframe, col_extract, win_length, overlap, True
    )
    labels = feature_extraction.generate_labels(
        user_id_dataframe, "Label", win_length, overlap
    )
    features["Label"] = labels
    new_user_id_dataframes.append(features)

d:\Work\HAR-Comparison-Privacy\src\utils\feature_extraction.py:56: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  labels = np.apply_along_axis(lambda x: stats.mode(x).mode, 1, labels)
d:\Work\HAR-Comparison-Privacy\src\utils\feature_extraction.py:56: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or

In [32]:
# Saving the data
processed_path = r"../processed_data/activity_recognition_wisdm/"
all_users_data = pd.DataFrame()
if not os.path.exists(processed_path):
    os.makedirs(processed_path)
for i in range(len(new_user_id_dataframes)):
    new_user_id_dataframes[i].insert(0, "User_ID", i + 1)
    new_user_id_dataframes[i].to_csv(
        processed_path + "user_" + str(i + 1) + ".csv", index=False
    )
    all_users_data = pd.concat([all_users_data, new_user_id_dataframes[i]])
all_users_data.to_csv(processed_path + "all_users.csv", index=False)